In [45]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import functools , os , shutil
import adanet
import tensorflow as tf
# The random seed to use.
RANDOM_SEED = 42

## [Medium 글](https://towardsdatascience.com/googles-adanet-uses-neural-networks-to-build-better-neural-networks-36c07eefd4d3)

## [Github TUTORIAL](https://github.com/tensorflow/adanet/blob/master/adanet/examples/tutorials/adanet_objective.ipynb)

In [46]:
tf.__version__

'1.15.0'

In [47]:
LOG_DIR = '/tmp/testmodels'

In [48]:
#@test {"skip": true}

get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

# Install ngrok binary.
# ! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
# ! unzip ngrok-stable-linux-amd64.zip

# Delete old logs dir.
shutil.rmtree(LOG_DIR, ignore_errors=True)

# print("Follow this link to open TensorBoard in a new tab.")
# get_ipython().system_raw('./ngrok http 6006 &')
# ! curl -s http://localhost:4040/api/tunnels | python3 -c \
#     "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

In [49]:
from sklearn.model_selection import train_test_split 
import pandas as pd

In [50]:
data = pd.read_csv("./Data/bank.csv")
data.isnull().sum()

age          0
job          0
marital      0
education    0
default      0
balance      0
housing      0
loan         0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [51]:
data = pd.read_csv("./Data/bank.csv")
inttype =data.select_dtypes("int").columns.tolist()
data[inttype] = data[inttype].astype(float)
target = data.pop("y")
target = pd.Categorical(target).codes
x_train , x_test , y_train  , y_test = train_test_split(data , target.reshape(-1,1))

# Preview the first example from the training data
print('Model inputs: %s \n' % x_train.iloc[0])
print('Model output: %s ' % (y_train[0]))

Model inputs: age                  37
job          technician
marital          single
education     secondary
default             yes
balance             375
housing              no
loan                 no
contact         unknown
day                   9
month               jun
duration            223
campaign              2
pdays                -1
previous              0
poutcome        unknown
Name: 1508, dtype: object 

Model output: [0] 


In [52]:
x_train.shape , x_test.shape , y_train.shape

((3390, 16), (1131, 16), (3390, 1))

In [53]:
INPUT_DIM = x_train.shape[1]
print(INPUT_DIM)

16


In [54]:
from tensorflow import feature_column

In [55]:
feature_columns = []
colinfo = dict(zip(data.dtypes.index ,data.dtypes.values))
for key , item in colinfo.items() :
    if item == "object" :
        uniitem = data[key].unique().tolist()
        cv = feature_column.categorical_column_with_vocabulary_list(key = key, 
                                                                    vocabulary_list=uniitem,
                                                                   )
        thal_one_hot = feature_column.indicator_column(cv)
        #feature_column.embedding_column(cv,dimension=3)
        feature_columns.append( thal_one_hot )
    else :
        feature_columns.append( feature_column.numeric_column(key) )

In [91]:
colinfo

{'age': dtype('float64'),
 'job': dtype('O'),
 'marital': dtype('O'),
 'education': dtype('O'),
 'default': dtype('O'),
 'balance': dtype('float64'),
 'housing': dtype('O'),
 'loan': dtype('O'),
 'contact': dtype('O'),
 'day': dtype('float64'),
 'month': dtype('O'),
 'duration': dtype('float64'),
 'campaign': dtype('float64'),
 'pdays': dtype('float64'),
 'previous': dtype('float64'),
 'poutcome': dtype('O')}

In [56]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns , trainable=True )

In [57]:
# features  = dict()
# for col_name in x_train.columns:
#     features[col_name] = x_train[col_name].tolist()
# dataset = tf.data.Dataset.from_tensor_slices((features , y_train))
# batch_size = 10
# dataset = dataset.batch(batch_size)
# iterator = dataset.make_one_shot_iterator()
# features, labels = iterator.get_next()
# input_layer = feature_layer(features)
# print(input_layer)

In [58]:
# last_layer = input_layer
# _layer_size = 10 
# kernel_initializer = tf.glorot_uniform_initializer(seed=1234)
# for _ in range(5):
#     print(last_layer)
#     last_layer = tf.layers.dense(
#       last_layer,
#       units= _layer_size,
#       activation=tf.nn.relu,
#       kernel_initializer=kernel_initializer)

In [59]:
_CSV_COLUMN_DEFAULTS =[ [""] if types == 'object' else [0.0] for types in data.dtypes.values.tolist() ]
_CSV_COLUMNS = data.columns.tolist()

In [60]:
FEATURES_KEY = "x"

def input_fn(partition, training, batch_size):
    """Generate an input function for the Estimator."""
    def _input_fn():
        if partition == "train":
            train_features  = dict()
            for col_name in x_train.columns:
                train_features[col_name] = x_train[col_name].tolist()
            dataset = tf.data.Dataset.from_tensor_slices((train_features , y_train))
        else:
            test_features  = dict()
            for col_name in x_test.columns:
                test_features[col_name] = x_test[col_name].tolist()
            dataset = tf.data.Dataset.from_tensor_slices((test_features , y_test))
    # We call repeat after shuffling, rather than before, to prevent separate
    # epochs from blending together.
        if training:
            dataset = dataset.shuffle(10 * batch_size, seed=RANDOM_SEED).repeat()

        dataset = dataset.batch(batch_size)
        iterator = dataset.make_one_shot_iterator()
        features, labels = iterator.get_next()
        return features , labels

    return _input_fn

In [61]:
_NUM_LAYERS_KEY = "num_layers"


class _SimpleDNNBuilder(adanet.subnetwork.Builder):
  """Builds a DNN subnetwork for AdaNet."""

  def __init__(self, optimizer, layer_size, num_layers, learn_mixture_weights,
               seed):
    """Initializes a `_DNNBuilder`.

    Args:
      optimizer: An `Optimizer` instance for training both the subnetwork and
        the mixture weights.
      layer_size: The number of nodes to output at each hidden layer.
      num_layers: The number of hidden layers.
      learn_mixture_weights: Whether to solve a learning problem to find the
        best mixture weights, or use their default value according to the
        mixture weight type. When `False`, the subnetworks will return a no_op
        for the mixture weight train op.
      seed: A random seed.

    Returns:
      An instance of `_SimpleDNNBuilder`.
    """

    self._optimizer = optimizer
    self._layer_size = layer_size
    self._num_layers = num_layers
    self._learn_mixture_weights = learn_mixture_weights
    self._seed = seed

  def build_subnetwork(self,
                       features,
                       logits_dimension,
                       training,
                       iteration_step,
                       summary,
                       previous_ensemble=None):
    """
    See `adanet.subnetwork.Builder`.
    이 부분을 바꿔줘야하나?? 
    """
#     input_layer = tf.to_float(features[FEATURES_KEY])
    input_layer = feature_layer( features)
    kernel_initializer = tf.glorot_uniform_initializer(seed=self._seed)
    last_layer = input_layer
    for _ in range(self._num_layers):
      last_layer = tf.layers.dense(
          last_layer,
          units=self._layer_size,
          activation=tf.nn.relu,
          kernel_initializer=kernel_initializer)
    logits = tf.layers.dense(
        last_layer,
        units=logits_dimension,
        kernel_initializer=kernel_initializer)

    persisted_tensors = {_NUM_LAYERS_KEY: tf.constant(self._num_layers)}
    return adanet.Subnetwork(
        last_layer=last_layer,
        logits=logits,
        complexity=self._measure_complexity(),
        persisted_tensors=persisted_tensors)

  def _measure_complexity(self):
    """Approximates Rademacher complexity as the square-root of the depth."""
    return tf.sqrt(tf.to_float(self._num_layers))

  def build_subnetwork_train_op(self, subnetwork, loss, var_list, labels,
                                iteration_step, summary, previous_ensemble):
    """See `adanet.subnetwork.Builder`."""
    return self._optimizer.minimize(loss=loss, var_list=var_list)

  def build_mixture_weights_train_op(self, loss, var_list, logits, labels,
                                     iteration_step, summary):
    """See `adanet.subnetwork.Builder`."""

    if not self._learn_mixture_weights:
      return tf.no_op()
    return self._optimizer.minimize(loss=loss, var_list=var_list)

  @property
  def name(self):
    """See `adanet.subnetwork.Builder`."""

    if self._num_layers == 0:
      # A DNN with no hidden layers is a linear model.
      return "linear"
    return "{}_layer_dnn".format(self._num_layers)


class SimpleDNNGenerator(adanet.subnetwork.Generator):
  """Generates a two DNN subnetworks at each iteration.

  The first DNN has an identical shape to the most recently added subnetwork
  in `previous_ensemble`. The second has the same shape plus one more dense
  layer on top. This is similar to the adaptive network presented in Figure 2 of
  [Cortes et al. ICML 2017](https://arxiv.org/abs/1607.01097), without the
  connections to hidden layers of networks from previous iterations.
  """

  def __init__(self,
               optimizer,
               layer_size=64,
               learn_mixture_weights=False,
               seed=None):
    """Initializes a DNN `Generator`.

    Args:
      optimizer: An `Optimizer` instance for training both the subnetwork and
        the mixture weights.
      layer_size: Number of nodes in each hidden layer of the subnetwork
        candidates. Note that this parameter is ignored in a DNN with no hidden
        layers.
      learn_mixture_weights: Whether to solve a learning problem to find the
        best mixture weights, or use their default value according to the
        mixture weight type. When `False`, the subnetworks will return a no_op
        for the mixture weight train op.
      seed: A random seed.

    Returns:
      An instance of `Generator`.
    """

    self._seed = seed
    self._dnn_builder_fn = functools.partial(
        _SimpleDNNBuilder,
        optimizer=optimizer,
        layer_size=layer_size,
        learn_mixture_weights=learn_mixture_weights)

  def generate_candidates(self, previous_ensemble, iteration_number,
                          previous_ensemble_reports, all_reports):
    """See `adanet.subnetwork.Generator`."""

    num_layers = 0
    seed = self._seed
    if previous_ensemble:
      num_layers = tf.contrib.util.constant_value(
          previous_ensemble.weighted_subnetworks[
              -1].subnetwork.persisted_tensors[_NUM_LAYERS_KEY])
    if seed is not None:
      seed += iteration_number
    return [
        self._dnn_builder_fn(num_layers=num_layers, seed=seed),
        self._dnn_builder_fn(num_layers=num_layers + 1, seed=seed),
    ]

In [62]:
LEARNING_RATE = 0.001  #@param {type:"number"}
TRAIN_STEPS = 50000  #@param {type:"integer"}
BATCH_SIZE = 300  #@param {type:"integer"}
LEARN_MIXTURE_WEIGHTS = False  #@param {type:"boolean"}
ADANET_LAMBDA = 0  #@param {type:"number"}
ADANET_ITERATIONS = 3  #@param {type:"integer"}

def train_and_evaluate(experiment_name, learn_mixture_weights=LEARN_MIXTURE_WEIGHTS,
                       adanet_lambda=ADANET_LAMBDA):
    """Trains an `adanet.Estimator` to predict housing prices."""

    model_dir = os.path.join(LOG_DIR, experiment_name)
    print(model_dir)
    estimator = adanet.Estimator(
      # Since we are predicting housing prices, we'll use a regression
      # head that optimizes for MSE.
      ### change 
#      head = tf.contrib.estimator.multi_class_head(
#         NUM_CLASSES, loss_reduction=loss_reduction)
      head=tf.contrib.estimator.binary_classification_head(
          loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE),
        model_dir  = model_dir , 
      # Define the generator, which defines our search space of subnetworks
      # to train as candidates to add to the final AdaNet model.
      subnetwork_generator=SimpleDNNGenerator(
          optimizer=tf.train.RMSPropOptimizer(learning_rate=LEARNING_RATE),
          learn_mixture_weights=learn_mixture_weights,
          seed=RANDOM_SEED),

      # Lambda is a the strength of complexity regularization. A larger
      # value will penalize more complex subnetworks.
      adanet_lambda=adanet_lambda,

      # The number of train steps per iteration.
      max_iteration_steps=TRAIN_STEPS // ADANET_ITERATIONS,

      # The evaluator will evaluate the model on the full training set to
      # compute the overall AdaNet loss (train loss + complexity
      # regularization) to select the best candidate to include in the
      # final AdaNet model.
      evaluator=adanet.Evaluator(
          input_fn=input_fn("train", training=False, batch_size=BATCH_SIZE)),

      # Configuration for Estimators.
      config=tf.estimator.RunConfig(
          save_summary_steps=5000,
          save_checkpoints_steps=5000,
          tf_random_seed=RANDOM_SEED,
          model_dir=model_dir) 
    )

  # Train and evaluate using using the tf.estimator tooling.
    train_spec = tf.estimator.TrainSpec(
        input_fn=input_fn("train", training=True, batch_size=BATCH_SIZE),
        max_steps=TRAIN_STEPS)
    eval_spec = tf.estimator.EvalSpec(
        input_fn=input_fn("test", training=False, batch_size=BATCH_SIZE),
        steps=None,
        start_delay_secs=1,
        throttle_secs=30,
  )
    return [tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec) , estimator]


def ensemble_architecture(result):
    """Extracts the ensemble architecture from evaluation results."""

    architecture = result["architecture/adanet/ensembles"]
    # The architecture is a serialized Summary proto for TensorBoard.
    summary_proto = tf.summary.Summary.FromString(architecture)
    return summary_proto.value[0].tensor.string_val[0]

In [63]:
# results, _ , estimator
results = train_and_evaluate("uniform_average_numcat_classifier_ensemble_baseline")
# print("Loss:", results[0]["average_loss"])
# print("Architecture:", ensemble_architecture(results[0]))

W1229 21:16:26.886602 139997072619264 estimator.py:653] The following arguments have been moved to `adanet.ensemble.ComplexityRegularizedEnsembler` which can be specified in the `ensemblers` argument: ['adanet_lambda']


/tmp/testmodels/uniform_average_numcat_classifier_ensemble_baseline


W1229 21:16:28.281506 139997072619264 ensemble_builder.py:490] The `build_mixture_weights_train_op` method is deprecated. Please use the `Ensembler#build_train_op` instead.
W1229 21:16:28.410232 139997072619264 ensemble_builder.py:490] The `build_mixture_weights_train_op` method is deprecated. Please use the `Ensembler#build_train_op` instead.
W1229 21:17:56.827423 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:17:56.846445 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:17:57.591434 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:17:57.609939 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" ins

TypeError: tuple indices must be integers or slices, not str

In [66]:
results[0]

({'accuracy': 0.8859416,
  'accuracy_baseline': 0.88771,
  'architecture/adanet/ensembles': b'\n6\n\x13architecture/adanetB\x15\x08\x07\x12\x00B\x0f| 1_layer_dnn |J\x08\n\x06\n\x04text',
  'auc': 0.8219014,
  'auc_precision_recall': 0.43094778,
  'average_loss': 0.5164742,
  'best_ensemble_index_0': 1,
  'iteration': 0,
  'label/mean': 0.11229001,
  'loss': 0.5056766,
  'precision': 0.4909091,
  'prediction/mean': 0.109021515,
  'recall': 0.42519686,
  'global_step': 16666},
 [])

In [67]:
output = results[1].predict(input_fn= input_fn("train", training=False, batch_size=1))
output = list(output)

W1229 21:35:55.692458 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:35:55.730070 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:35:56.431041 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:35:56.449331 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:35:57.326519 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:35:57.346177 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:3

In [69]:
pd.DataFrame(output)

,all_class_ids,all_classes,class_ids,classes,logistic,logits,probabilities
0,"[0, 1]","[b'0', b'1']",[0],[b'0'],[2.2796834e-07],[-15.294059],"[0.99999976, 2.2796836e-07]"
1,"[0, 1]","[b'0', b'1']",[0],[b'0'],[0.0],[-83.04176],"[1.0, 8.618277e-37]"
2,"[0, 1]","[b'0', b'1']",[1],[b'1'],[0.98426825],[4.136216],"[0.01573177, 0.98426825]"
3,"[0, 1]","[b'0', b'1']",[1],[b'1'],[0.99246275],[4.8803377],"[0.0075372076, 0.99246275]"
4,"[0, 1]","[b'0', b'1']",[0],[b'0'],[0.0],[-19.424355],"[1.0, 3.6653038e-09]"
...,...,...,...,...,...,...,...
3385,"[0, 1]","[b'0', b'1']",[0],[b'0'],[0.0],[-46.153744],"[1.0, 9.029917e-21]"
3386,"[0, 1]","[b'0', b'1']",[0],[b'0'],[2.2541274e-08],[-17.607918],"[1.0, 2.2541274e-08]"
3387,"[0, 1]","[b'0', b'1']",[0],[b'0'],[0.0],[-32.932728],"[1.0, 4.983082e-15]"
3388,"[0, 1]","[b'0', b'1']",[0],[b'0'],[0.0022059528],[-6.1143875],"[0.99779403, 0.0022059523]"


In [71]:
tf.feature_column.make_parse_example_spec( feature_columns )

{'age': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None),
 'job': VarLenFeature(dtype=tf.string),
 'marital': VarLenFeature(dtype=tf.string),
 'education': VarLenFeature(dtype=tf.string),
 'default': VarLenFeature(dtype=tf.string),
 'balance': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None),
 'housing': VarLenFeature(dtype=tf.string),
 'loan': VarLenFeature(dtype=tf.string),
 'contact': VarLenFeature(dtype=tf.string),
 'day': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None),
 'month': VarLenFeature(dtype=tf.string),
 'duration': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None),
 'campaign': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None),
 'pdays': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None),
 'previous': FixedLenFeature(shape=(1,), dtype=tf.float32, default_value=None),
 'poutcome': VarLenFeature(dtype=tf.string)}

In [72]:
#input_column = tf.feature_column.numeric_column("x", shape=(INPUT_DIM,))
serving_input_fn = tf.estimator.export.build_parsing_serving_input_receiver_fn(
  tf.feature_column.make_parse_example_spec( feature_columns ))

In [73]:
model_dir = '/tmp/testmodels/uniform_average_numcat_classifier_ensemble_baseline/'
results[1].export_saved_model(export_dir_base = model_dir , 
                              serving_input_receiver_fn = serving_input_fn ,
                              checkpoint_path = None , 
                              experimental_mode='infer'
                             )

W1229 21:37:46.315855 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:37:46.336557 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:37:46.877105 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:37:46.895499 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:37:47.753273 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:37:47.772034 139997072619264 metrics_impl.py:803] Trapezoidal rule is known to produce incorrect PR-AUCs; please switch to "careful_interpolation" instead.
W1229 21:3

b'/tmp/testmodels/uniform_average_numcat_classifier_ensemble_baseline/1577623078'

In [74]:
from tensorflow.contrib import predictor
path = '/tmp/testmodels/uniform_average_numcat_classifier_ensemble_baseline/1577623078'
predict_fn = predictor.from_saved_model(path)

W1229 21:38:10.685141 139997072619264 deprecation.py:323] From /root/anaconda3/lib/python3.6/site-packages/tensorflow_core/contrib/predictor/saved_model_predictor.py:153: load (from tensorflow.python.saved_model.loader_impl) is deprecated and will be removed in a future version.
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.


In [75]:
predict_fn

SavedModelPredictor with feed tensors {'inputs': <tf.Tensor 'input_example_tensor:0' shape=(?,) dtype=string>} and fetch_tensors {'scores': <tf.Tensor 'adanet/iteration_3/best_export_outputs/strided_slice_3:0' shape=(?, 2) dtype=float32>, 'classes': <tf.Tensor 'adanet/iteration_3/best_export_outputs/strided_slice_4:0' shape=(?, 2) dtype=string>}

In [76]:
from tensorflow.contrib import slim
path = '/tmp/testmodels/uniform_average_numcat_classifier_ensemble_baseline/1577623078'
#Model = tf.saved_model.load(path, )
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.load(sess, ["serve"], export_dir= path)
    graph = tf.get_default_graph()
    print(graph.get_operations())

[<tf.Operation 'global_step/Initializer/zeros' type=Const>, <tf.Operation 'global_step' type=VariableV2>, <tf.Operation 'global_step/Assign' type=Assign>, <tf.Operation 'global_step/read' type=Identity>, <tf.Operation 'input_example_tensor' type=Placeholder>, <tf.Operation 'ParseExample/Const' type=Const>, <tf.Operation 'ParseExample/Const_1' type=Const>, <tf.Operation 'ParseExample/Const_2' type=Const>, <tf.Operation 'ParseExample/Const_3' type=Const>, <tf.Operation 'ParseExample/Const_4' type=Const>, <tf.Operation 'ParseExample/Const_5' type=Const>, <tf.Operation 'ParseExample/Const_6' type=Const>, <tf.Operation 'ParseExample/ParseExample/names' type=Const>, <tf.Operation 'ParseExample/ParseExample/sparse_keys_0' type=Const>, <tf.Operation 'ParseExample/ParseExample/sparse_keys_1' type=Const>, <tf.Operation 'ParseExample/ParseExample/sparse_keys_2' type=Const>, <tf.Operation 'ParseExample/ParseExample/sparse_keys_3' type=Const>, <tf.Operation 'ParseExample/ParseExample/sparse_keys_4'

In [24]:
import numpy as np

In [90]:
def _bytes_feature(value):
    """string / byte 타입을 받아서 byte list를 리턴합니다."""
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """float / double 타입을 받아서 float list를 리턴합니다."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """bool / enum / int / uint 타입을 받아서 int64 list를 리턴합니다."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

In [100]:
objcol = x_train.select_dtypes("object").columns.tolist()
numcol = x_train.select_dtypes("float").columns.tolist()

In [112]:
print(objcol)
print(numcol)

['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']
['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']


In [119]:
print(x_test.columns.tolist())

['age', 'job', 'marital', 'education', 'default', 'balance', 'housing', 'loan', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome']


In [110]:
exported_path = '/tmp/testmodels/uniform_average_numcat_classifier_ensemble_baseline/1577623078'
predictor= tf.contrib.predictor.from_saved_model(exported_path)
for idx , line in  x_train.iterrows() :
    feature = {}
    for col in line.index.values.tolist() :
        if col in objcol :
            feature[col] = _bytes_feature(line[col].encode('utf-8'))
        elif col in numcol :
            feature[col] = _float_feature(line[col])
    model_input = tf.train.Example(features=tf.train.Features(feature=feature)) 
    model_input=model_input.SerializeToString()
    output_dict= predictor({"inputs":[model_input]})
    print(output_dict)

{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[9.9999976e-01, 2.2796836e-07]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.000000e+00, 8.618277e-37]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[0.01573177, 0.98426825]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[0.00753721, 0.99246275]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 3.6653038e-09]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.0000000e+00, 9.5054776e-32]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1., 0.]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[1.000000e+00, 3.311321e-12]], dtype=float32)}
{'classes': array([[b'0', b'1']], dtype=object), 'scores': array([[0.99807525, 0.00192473]], dtype=float32)}
{'classes

In [22]:
def save_model(adanet_estimator, model_dir, input_dimension):
    def serving_input_fn():
        inputs = {
            "x": tf.placeholder(dtype=tf.float32,
                                shape=[None, input_dimension],
                                name="x")
        }
        return tf.estimator.export.ServingInputReceiver(inputs, inputs)
    return adanet_estimator.estimator.export_saved_model(model_dir, serving_input_fn)

def print_model_architechture(model_dir):
    with tf.Session(graph=tf.Graph()) as sess:
        tf.saved_model.load(sess, ["serve"], model_dir)
        model_vars = tf.trainable_variables()
        slim.model_analyzer.analyze_vars(model_vars, print_info=True)